In [ ]:
import cv2
import numpy as np
import torch
from skimage.color import rgb2yuv,yuv2rgb
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
from model import generator
from torch.autograd import Variable
from scipy.ndimage import zoom

G = generator()
G.cuda()
checkpoint = torch.load('.\\G.pth', map_location='cuda:0')
G.load_state_dict(checkpoint)

p = Image.open('..\\..\\train_data\\Places365_val_00000091.jpg').convert('RGB')
arr = np.array(p.convert('L'))
img_yuv = rgb2yuv(p)
H,W,_ = img_yuv.shape
infimg = np.expand_dims(np.expand_dims(img_yuv[...,0], axis=0), axis=0)
img_variable = Variable(torch.Tensor(infimg-0.5)).cuda()

res = G(img_variable)
uv = res.cpu().detach().numpy()
uv[:,0,:,:] *= 0.436
uv[:,1,:,:] *= 0.615
(_,_,H1,W1) = uv.shape
uv = zoom(uv,(1,1,H/H1,W/W1))
yuv = np.concatenate([infimg,uv],axis=1)[0]
rgb = yuv2rgb(yuv.transpose(1,2,0))
final = (rgb.clip(min=0,max=1)*256)[:,:,[2,1,0]]

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(10, 10))
axs[0].set_title('Grayscale Image')
axs[0].imshow(arr, cmap='gray', vmin=0, vmax=255)
axs[1].set_title('Colorized Image')
axs[1].imshow(rgb)
axs[2].set_title('Original Image')
axs[2].imshow(p)